In [2]:
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from sklearn.cluster import KMeans

import ipywidgets as widgets
from ipywidgets import interact, IntSlider, fixed

# --- User options ---
img_path = 'Van.jpg'

# --- Load image to get its size for slider limits ---
img = Image.open(img_path)
img_width, img_height = img.size

def process_patch(
    xmin=100, xmax=800, ymin=100, ymax=300,
    n_colors=4,
    target_height=24, target_width=100
):
    # Clamp values to image size
    xmin = max(0, min(xmin, img_width-1))
    xmax = max(xmin+1, min(xmax, img_width))
    ymin = max(0, min(ymin, img_height-1))
    ymax = max(ymin+1, min(ymax, img_height))
    target_res = (target_width, target_height)

    display(img)
    print(f"Image size: {img.size} (width, height)")
    print(f"Selected rectangle: ({xmin}, {ymin}) to ({xmax}, {ymax})")
    print(f"Target resolution: {target_res}")
    print(f"Number of colors: {n_colors}")

    # --- Crop the patch from the image ---
    patch = img.crop((xmin, ymin, xmax, ymax))

    # --- Downsample the patch to target resolution ---
    patch_downsampled = patch.resize(target_res, Image.LANCZOS)

    # --- Convert downsampled patch to numpy array ---
    patch_np = np.array(patch_downsampled)
    h, w = patch_np.shape[:2]
    patch_reshaped = patch_np.reshape(-1, 3)

    # --- Use KMeans to find n_colors basic colors ---
    kmeans = KMeans(n_clusters=n_colors, random_state=0, n_init=10)
    labels = kmeans.fit_predict(patch_reshaped)
    basic_colors = kmeans.cluster_centers_.astype(np.uint8)

    # --- Recreate the patch using the basic colors ---
    approx_patch = basic_colors[labels].reshape(h, w, 3)

    # --- Show the original and approximated patch side by side ---
    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    axs[0].imshow(patch_downsampled)
    axs[0].set_title(f"Downsampled Patch ({w}x{h})")
    axs[0].axis('off')
    axs[1].imshow(approx_patch)
    axs[1].set_title(f"Approximated with {n_colors} Colors")
    axs[1].axis('off')
    plt.show()

    # --- Optionally, show the rectangle on the original image ---
    fig, ax = plt.subplots()
    ax.imshow(img)
    rect_patch = patches.Rectangle(
        (xmin, ymin), xmax - xmin, ymax - ymin,
        linewidth=2, edgecolor='r', facecolor='none'
    )
    ax.add_patch(rect_patch)
    plt.show()

# --- Interactive sliders for parameters ---
interact(
    process_patch,
    xmin=IntSlider(min=0, max=img_width-2, step=1, value=100, description='xmin'),
    xmax=IntSlider(min=1, max=img_width, step=1, value=800, description='xmax'),
    ymin=IntSlider(min=0, max=img_height-2, step=1, value=100, description='ymin'),
    ymax=IntSlider(min=1, max=img_height, step=1, value=300, description='ymax'),
    n_colors=IntSlider(min=2, max=16, step=1, value=4, description='n_colors'),
    target_height=IntSlider(min=8, max=img_height, step=1, value=24, description='height'),
    target_width=IntSlider(min=8, max=img_width, step=1, value=100, description='width')
)

interactive(children=(IntSlider(value=100, description='xmin', max=1198), IntSlider(value=800, description='xm…

<function __main__.process_patch(xmin=100, xmax=800, ymin=100, ymax=300, n_colors=4, target_height=24, target_width=100)>